In [1]:
import pandas as pd
import re

In [2]:
def get_senioridade(df):
    mapa_senioridade = {
        'Júnior': ['Júnior', 'Junior', 'Jr', 'Estágio', 'Estagiário', 'Trainee', 'trainne'],
        'Pleno': ['Pleno', 'Plena', 'Pl', 'Jr/Pl'],
        'Sênior': ['Sênior', 'Senior', 'Sr', 'Pl/Sr', 'Tech Lead', 'Líder']
    }

    for index, row in df.iterrows():
        for senioridade, termos in mapa_senioridade.items():
            for termo in termos:
                if re.search(r'\b' + termo + r'\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                   re.search(r'\b' + termo + r'\b', row['descricao'], flags=re.IGNORECASE):
                    df.at[index, 'senioridade'] = senioridade
                    break

    return df

In [3]:
def format_benefits_list(description: str) -> list[str] | None:
    """Extrai e padroniza os benefícios a partir da descrição da vaga.

    Args:
        description (str): Descrição da vaga.

    Returns:
        list[str] | None: Lista com os benefícios padronizados, ou None se não forem encontrados.

    Exemplos:
    ---
    >>> format_benefits_list('Oferecemos vale-alimentação e plano de saúde.')
    ['Vale-alimentação', 'Plano de saúde']
    """

    try:
        # Inicializa uma lista para armazenar os benefícios extraídos
        extracted_benefits = []

        # Converte toda a descrição para minúsculas para facilitar a correspondência
        description_lower = description.lower()

        # Percorre o mapa de benefícios para verificar a presença de cada benefício na descrição
        for benefit, synonyms in benefit_map.items():
            # Verifica se algum sinônimo do benefício está presente na descrição
            if any(synonym.lower() in description_lower for synonym in synonyms):
                extracted_benefits.append(benefit)

        # Retorna a lista de benefícios extraídos, removendo duplicatas e mantendo a ordem original
        if extracted_benefits:
            return list(dict.fromkeys(extracted_benefits))
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [4]:
benefit_map = {
    'Assistência médica':                    ['medica', 'saude', 'medico'],
    'Assistência odontológica':              ['odonto', 'dentista'],
    'Assistência psicológia':                ['psicolog', 'saude mental'],
    'Auxílio academia':                      ['academia', 'gympass', 'gym', 'total pass'],
    'Auxílio combustível':                   ['combustivel'],
    'Auxílio creche':                        ['creche'],
    'Auxílio desenvolvimento':               ['desenvolvimento'],
    'Auxílio estacionamento':                ['estacionamento'],
    'Auxílio farmácia':                      ['farmacia', 'medicamento'],
    'Auxílio fretado':                       ['fretado'],
    'Auxílio home office':                   ['auxilio home', 'custo home'],
    'Bicicletário':                          ['bicleta'],
    'Bolsa auxílio':                         ['bolsa auxilio'],
    'Café da manhã':                         ['cafe da manha'],
    'Cesta básica':                          ['cesta basica'],
    'Cesta de natal':                        ['natal'],
    'Clube de vantagens':                    ['vantagens'],
    'Consignado':                            ['consignado'],
    'Convênio com empresas parceiras':       ['convenio'],
    'Cooperativa de crédito':                ['cooperativa de credito'],
    'Day off aniversário':                   ['aniversario'],
    'Desconto em produtos':                  ['desconto em produtos'],
    'Ginástica laboral':                     ['ginastica laboral'],
    'Horário flexível':                      ['horario flexivel', 'flexibilidade'],
    'Massoterapia':                          ['massoterapia', 'massagem'],
    'Participação nos Lucros ou Resultados': ['lucros', 'plr', 'ppr'],
    'Plano de Aquisição de Ações':           ['aquisicao de acoes'],
    'Previdência privada':                   ['previdencia'],
    'Programa de remuneração variável':      ['remuneracao variavel', 'bonificac', 'premiac'],
    'Programa de treinamentos':              ['treinamento', 'capacitacao'],
    'Refeitório':                            ['refeitorio'],
    'Restaurante interno':                   ['restaurante'],
    'Sala de Jogos':                         ['jogos'],
    'Seguro de vida':                        ['vida'],
    'Vale-alimentação':                      ['alimentacao'],
    'Vale-cultura':                          ['vale cultura', 'vale-cultura'],
    'Vale-refeição':                         ['refeicao', 'ticket'],
    'Vale-transporte':                       ['transporte'],
}

In [5]:
df = pd.read_csv('../data/data_raw/vagas_glassdoor_raw.csv')

In [6]:
df = get_senioridade(df)
df['senioridade'].replace('NaN', 'Não informado')
df['modalidade'] = df['modalidade'].replace('trabalho remoto', 'Remoto')
df['estado'] = df['estado'].replace('Distrito Federal', 'DF')
df.loc[df['modalidade'] == 'Remoto', 'cidade'] = 'Não informado'
df.loc[df['modalidade'] == 'Remoto', 'estado'] = 'Todo o Brasil'
df['regime'] = 'CLT'

df['beneficios'] = df['descricao'].apply(format_benefits_list)

In [7]:
df

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills,beneficios
0,Glassdoor,https://www.glassdoor.com.br/job-listing/057-p...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-16,NaN,2024-02-23,Cientista de dados,Pleno,[057] Pessoa Engenheira de Dados Plena,ilegra,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009130217500,Você é uma pessoa apaixonada pelo que faz e bu...,"['Azure', 'NoSQL', 'Git', 'English', 'SQL', 'M...","[Assistência odontológica, Auxílio academia, H..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-19,NaN,2024-02-23,Cientista de dados,NaN,0726 - analista de dados - sede administrativa...,Netvagas,Palotina,PR,NaN,NaN,CLT,NaN,1009134443006,Descrição:\nHabilidades necessárias:\nSuperior...,"['Statistics', 'Computer science', 'Power BI',...","[Auxílio desenvolvimento, Seguro de vida]"
2,Glassdoor,https://www.glassdoor.com.br/job-listing/0726-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-09,NaN,2024-02-23,Cientista de dados,NaN,0726 - Analista de Dados - Sede Administrativa...,Sicredi,Palotina,PR,NaN,NaN,CLT,NaN,1008695489204,Somos uma instituição que está constantemente ...,"['Statistics', 'Computer science', 'Power BI',...","[Assistência odontológica, Auxílio academia, A..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/3229-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-01-30,NaN,2024-02-23,Cientista de dados,NaN,3229 Analista de dados,OEmprego,Porto Alegre,RS,NaN,NaN,CLT,NaN,1009098963816,"Salário: R$ 2.200,00\nBenefícios: VT\nRamo emp...","['Statistics', 'Computer science', 'Big data',...","[Programa de treinamentos, Seguro de vida]"
4,Glassdoor,https://www.glassdoor.com.br/job-listing/67414...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-14,NaN,2024-02-23,Cientista de dados,Sênior,6741487 - PESSOA ENGENHEIRA DE DADOS SÊNIOR - OKR,Quality Digital,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009126052954,Nós somos a Quality Digital! Saiba mais sobre ...,"['Azure', 'Business intelligence', 'Data struc...","[Assistência médica, Assistência odontológica,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,Glassdoor,https://www.glassdoor.com.br/job-listing/tech-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-02,NaN,2024-02-23,Cientista de dados,Sênior,Tech Lead Cientista de Dados,Big Data,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009055259074,"Somos a Big Data, líderes no Brasil em Intelig...","['TensorFlow', 'Big data', 'Machine learning',...","[Auxílio academia, Auxílio desenvolvimento, Au..."
745,Glassdoor,https://www.glassdoor.com.br/job-listing/tech-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-17,NaN,2024-02-23,Cientista de dados,Sênior,Tech lead cientista de dados,Netvagas,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009131627689,"Descrição:\nExperiência sólida em projetar, de...","['TensorFlow', 'Big data', 'Machine learning',...",None
746,Glassdoor,https://www.glassdoor.com.br/job-listing/train...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-01-27,NaN,2024-02-23,Cientista de dados,Júnior,Trainee em Ciência de Dados,Huork.com,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1008948636191,TRAINEE EM CIÊNCIA DE DADOS\n\nLOCAL: Remoto (...,"['Statistics', 'Power BI', 'R', 'QlikView', 'T...",None
747,Glassdoor,https://www.glassdoor.com.br/job-listing/vaga-...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-05,NaN,2024-02-23,Cientista de dados,Sênior,Vaga afirmativa para mulheres - Cientista de D...,DIGITIZEI INFORMATICA LTDA,Santo Antônio da Platina,PR,NaN,NaN,CLT,NaN,1009111917469,Somos uma startup dinâmica de contru-tech com ...,"['English', 'SQL', 'Docker', 'Machine learning...","[Auxílio desenvolvimento, Horário flexível]"


In [8]:
df.to_csv('../data/data_clean/vagas_glassdoor_clean.csv', index=False)